<center><img src="images/logo_CESI.png" width="250"></center>

# <center><b>Projet algorithmique et optimisation combinatoire</b></center>
# <center><b>Livrable 1</b></center>

<p><center><b><u>Groupe de projet</u> :</b></center></p>
<center><b>Mickaël King</b></center>
<center><b>(À compléter)</b></center>
<center><b>(À compléter)</b></center>
<center><b>(À compléter)</b></center>

<p><center><b><u>Tuteur</u> : Mohamed Belgacem</b></center></p>
<center><b><u>Promotion</u> : FISE A3 Informatique</b></center>

# Sommaire

1. [Contexte](#Contexte)
2. [Problématique](#Problematique)
3. [Propriété théorique](#Propriete-theorique)
4. [Contraintes](#Contraintes)
5. [Représentation des données](#Representation-des-donnees)
    * [Modélisation sous forme de graphe](#Modelisation-sous-forme-de-graphe)
    * [Modélisation des camions](#Modelisation-des-camions)
6. [Complexité du problème](#Complexite-du-probleme)
7. [Solutions et outils proposées](#Solutions-et-outils-proposees)
8. [Bibliographie](#Bibliographie)

# Contexte

(À compléter -> Mohamed)

# Problematique

(À compléter -> Ryan)

# Propriete theorique

(À compléter -> Oumar)

# Contraintes

(À compléter -> Mohamed)

# Representation des donnees

(À compléter -> Mickaël + les autres)

### Modelisation sous forme de graphe

D'un point de vue <b>théorique</b>, notre réseau de transport sera modéliser sous forme de <b>graphe</b>.

Nous partons du principe que chaque <b>villes</b> et représenter par un <b><u>sommet</u></b> et que chaque <b>liaison</b> entre ces villes seront représenter par une <b><u>arrête</u></b>.

<center><img src="images/representation_exemple.png" width="250"></center>

Dans l'exemple ci-dessus, nous avons 3 villes <b>A</b>, <b>B</b> et <b>C</b>.
En supposant qu'il n'y ait pas de sens unique, on peut passer de la ville <b>A</b> à la ville et de la ville <b>B</b> à la ville <b>C</b> et vice-versa.

Notre graphe pour modéliser le contexte et donc un <b><u>graphe non orienté</u></b>.

Quant à la contrainte des <b>points de collectes</b>, nous avons décidé d'attribuer pour chaque point de collecte spécifique à une ville différente. C'est à dire qu'un sommet représentant une ville peut aussi être considérer comme étant un point de collecte.

<center><img src="images/representation_exemple1.png" width="250"></center>

Par exemple dans l'exemple ci-dessus, le point de collecte se situe à la ville <b>C</b>.

On prendra en compte aussi que certain dépôt ne peut réapprovisionner que <b>certains type d'objets</b>. 

Par exemple si un camion qui livre seulement des peluches tombe à court de peluches à livrer, il ne pourra se réapprovisionner que dans un dépôt qui réapprovisionne des peluches.

### Modelisation des camions

Pour la contrainte de <b>flotte hétérogène</b>, nous imaginons que chaque camions sera stocker en mémoire dans une structure de donnée spécifique qui seront des <b>tableaux</b>.

Sachant que chaque camion ne peut livrer qu'un <b>seul type d'objet</b>, chaque tableaux devra contenir des camions pour un seul type d'objet spécifique uniquement.

Cela nous permettra à la fois <b>d'organiser</b> facilement les camions en fonction du type d'objet mais aussi de pouvoir aller <b>chercher</b> un camion spécifique en particulier en accedant tout simplement au tableau.

<center><img src="images/representation_exemple2.png" width="250"></center>

Dans l'exemple ci-dessus, nous imaginons que des camions qui ne transporte que des peluches seront tous ranger dans un même tableau en commun. Nous avons dans l'exemple 6 camions (A1, A2, ..., A6).

# Complexite du probleme

(À compléter -> Oumar + les autres)

# Solutions et outils proposees

(À compléter -> Ryan)

# Bibliographie

(À compléter -> Tout le monde)

Site qui explique comment implémenter des graphes et appliquer des algorithmes connues d'un point de vue informatique :
https://www.geeksforgeeks.org/dsa/graph-data-structure-and-algorithms/